SumPi: Blanchard, Aloise and DeSarbo in the Journal of Marketing Research, 2017
Python code by Zhengrong Gu

loadData, helper_fun, helper_OPPSP are vns_function are the required Python scripts.

In [17]:
import loadData
import pandas as pd
import numpy as np
import random
import helper_fun
import helper_OPPSP

def vns_function(filename,n,minK,maxK,runs,maxIterations,display_interval,random_seed,onePilePerSumPile):
    ok = True
    I = 1 
    # read file
    if filename[-3:] == "csv":
        ci,J,raw_data,weights,yilij = loadData.load_csv(filename,n)
    else:
        ci,J,raw_data,weights,yilij = loadData.load_xlsx(filename,n)

    # setting some results
    processed_data = {}
    processed_data['yilij'] = yilij
    processed_data['weights'] = weights

    #If onePilePerSumPile==true then the minimum number of summary piles is 
    #the highest number of piles created by individuals
    if onePilePerSumPile == True:
        yilij = raw_data['yilij']
        I = len(yilij)
        ci = np.zeros(I)
        for i in range(I):
            ci[i] = len(yilij[i])
        
        val = max(ci)
        cid = np.argmax(ci)
        if val>minK:
            print(val,minK)
            ok = False
            processed_data=-1
            results=-1
            return

    # start the algorithm
    # Setting some results and configurations and initializing cells
    results = {}
    results['seed'] = np.zeros(shape=(maxK-minK+1,runs))
    
    for K in range(minK,maxK+1):
        for run in range(runs):
            results['seed'][K-minK][run] = random_seed + run + runs*(K-minK)

    results['minK'] = minK
    results['maxK'] = maxK
    results['runs'] = runs
    a = [0]*runs
    pmj_list = []
    for i in range(minK,maxK+1):
        pmj_list.append(a)
    results['pmj'] = pmj_list
    results['ximli'] = pmj_list
    results['Z'] = np.zeros(shape=(maxK-minK+1,runs))
    results['nIterations'] = np.zeros(shape=(maxK-minK+1,runs))
    results['truePos'] = pmj_list
    results['trueNeg'] = pmj_list
    results['covRate'] = pmj_list

    log_msg = ''
    k_max = 20

    #Starting algorithm for each number of summary piles and each replication
    for K in range(minK,maxK+1):
        for run in range(runs):
            random.seed(results['seed'][K-minK][run])
            print('%s\n--- K: %d - Run: %d ---\n\n'%(log_msg,K,run))
            #Start VNS settings ( not time limit here)
            stop_condition = False
            k = 1
            nIterations = 1

            #Initial solution: summary piles (pmj) and linking variables (ximli) are set to random
            pmj = [np.random.randint(1,3,(K,J))-1]
            bestPmj = pmj
            ximli = [[]]*I
            for i in range(I):
                if onePilePerSumPile == True:
                    ximli[i] = np.zeros(shape=(K,int(ci[i])))
                    perm = np.random.permutation(K)[:int(ci[i])]
                    for li in range(int(ci[i])):
                        ximli[i][perm[li]][li] = 1
                else:
                    ximli[i] = np.zeros(shape=(K,ci))
                    for li in range(ci):
                        ximli[i][np.random.randint(K)][li] = 1
            bestXimli = ximli

            bestZ = np.inf
            #print("ximli", ximli)

            #Shaking and local search
            while(stop_condition==False):
                if onePilePerSumPile == True:
                    ##to write
                    print("shake")
                    ximli = helper_OPPSP.shaking(bestXimli,bestPmj,yilij,k)
                    print("local")
                    Z, pmj, ximli = helper_OPPSP.local_search(ximli,pmj,yilij)
                else:
                    ximli=helper_fun.shaking(bestXimli,bestPmj,yilij,k)
                    Z, pmj, ximli = helper_fun.local_search(ximli,pmj,yilij,weights)

                #Updating best solution
                if Z < bestZ:
                    bestZ = Z
                    bestPmj = pmj
                    bestXimli = ximli
                    print("Iteration: %d - Best solution: %d.\n"%(nIterations,bestZ))
                    k = 1
                else:
                    k = k%k_max+1

                if nIterations % display_interval == 0:
                    print("Iteration: %d - Best solution: %d.\n"%(nIterations,bestZ))

                nIterations = nIterations+1
                if nIterations >= maxIterations:
                    stop_condition = True

            #Check if the best solution cost is trully the cost of this solution
            if onePilePerSumPile == True:
                _, Z = helper_OPPSP.get_cost(bestPmj,yilij)
            else:
                _, Z= helper_fun.get_cost(bestPmj, yilij, weights)

            if Z != bestZ:
                print("ERROR")

            #Check if summary piles are equal to any pile
            if onePilePerSumPile == True:
                #print("OPPSP")
                print("check piles")
                text = helper_OPPSP.check_piles(bestPmj,yilij,bestXimli)
                print(text)
            else:
                text = helper_fun.check_piles(bestPmj,yilij,bestXimli,weights,raw_data['yilij'])
                print(text)
            
            #Check some statistics and set results
            if onePilePerSumPile == True:
                #print("OPPSP")
                print("rating")
                truePos,trueNeg,covRate = helper_OPPSP.rating(bestPmj,yilij,bestXimli)
            else:
                truePos,trueNeg,covRate = helper_fun.rating(bestPmj,yilij,bestXimli,weights)

            results['pmj'][K-minK][run] = bestPmj[0]
            if onePilePerSumPile == True:
                results['ximli'][K-minK][run] = bestXimli
            else:
                results['ximli'][K-minK][run] = bestXimli[0]

            results['Z'][K-minK][run] = bestZ
            results['nIterations'] = nIterations-1
            results['truePos'][K-minK][run] = truePos
            results['trueNeg'][K-minK][run] = trueNeg 
            results['covRate'][K-minK][run] = covRate       

    return results

In [26]:
# Run the function:
# vns_function(filename,n,minK,maxK,runs,maxIterations,display_interval,random_seed,onePilePerSumPile)
a = vns_function("csv9c8a_cleaned2.csv", #filename
                 1, #n
                 4, #minK
                 5, #maxK
                 1, #runs 
                 100, #maxIterations
                 10, #display_iterval
                 2, #random seed
                 False) #one pile per summary pile


--- K: 4 - Run: 0 ---


cp 1
3
[1, 2, 3] 3
(4, 568) 568.0
[[1. 1. 1. ... 1. 1. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 1. 0. ... 1. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 1.]]
4 29
568 568
Iteration: 1 - Best solution: 3841.

cp 0
7
[0, 1, 2, 0, 1, 2, 3] 3
(4, 568) 568.0
[[1. 1. 0. ... 1. 1. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 1. 0. ... 1. 1. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]]
4 29
568 568
Iteration: 2 - Best solution: 3834.

cp 0
7
[0, 1, 2, 0, 1, 2, 3] 3
(4, 568) 568.0
[[1. 1. 0. ... 1. 1. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 1. 0. ... 1. 1. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]]
4 29
568 568
Iteration: 3 - Best solution: 3814.

cp 2
3
[0, 2, 3] 3
(4, 568) 568.0
[[1. 1. 0. ... 1. 1. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0

(4, 568) 568.0
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [1. 1. 0. ... 1. 1. 1.]
 [0. 0. 1. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [1. 1. 0. ... 1. 0. 1.]
 [0. 0. 1. ... 0. 0. 0.]]
4 29
568 568
cp 2
3
[0, 2, 3] 3
cp 2
3
[0, 2, 3] 3
cp 0
7
[0, 1, 2, 0, 1, 2, 3] 3
cp 1
3
[1, 2, 3] 3
cp 2
3
[0, 2, 3] 3
cp 2
3
[0, 2, 3] 3
cp 2
3
[0, 2, 3] 3
cp 3
3
[0, 1, 3] 3
cp 0
7
[0, 1, 2, 0, 1, 2, 3] 3
cp 2
3
[0, 2, 3] 3
(4, 568) 568.0
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [1. 1. 0. ... 1. 1. 1.]
 [0. 0. 1. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [1. 1. 0. ... 1. 0. 1.]
 [0. 0. 1. ... 0. 0. 0.]]
4 29
568 568
cp 1
3
[1, 2, 3] 3
cp 2
3
[0, 2, 3] 3
cp 0
7
[0, 1, 2, 0, 1, 2, 3] 3
cp 1
3
[1, 2, 3] 3
cp 3
3
[0, 1, 3] 3
cp 2
3
[0, 2, 3] 3
cp 2
3
[0, 2, 3] 3
cp 2
3
[0, 2, 3] 3
cp 2
3
[0, 2, 3] 3
cp 3
3
[0, 1, 3] 3
cp 0
7
[0, 1, 2, 0, 1, 2, 3] 3
(4, 568) 568.0
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [1. 1. 0. ... 1. 1. 1.]
 [0

cp 2
3
[0, 2, 3] 3
cp 3
3
[0, 1, 3] 3
cp 0
7
[0, 1, 2, 0, 1, 2, 3] 3
cp 3
3
[0, 1, 3] 3
cp 1
3
[1, 2, 3] 3
cp 3
3
[0, 1, 3] 3
cp 3
3
[0, 1, 3] 3
cp 2
3
[0, 2, 3] 3
cp 3
3
[0, 1, 3] 3
cp 2
3
[0, 2, 3] 3
(4, 568) 568.0
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [1. 1. 0. ... 1. 1. 1.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 1. 0. 1.]
 [0. 1. 1. ... 0. 0. 0.]]
4 29
568 568
cp 3
3
[0, 1, 3] 3
cp 2
3
[0, 2, 3] 3
cp 3
3
[0, 1, 3] 3
cp 0
7
[0, 1, 2, 0, 1, 2, 3] 3
cp 1
3
[1, 2, 3] 3
cp 1
3
[1, 2, 3] 3
cp 1
3
[1, 2, 3] 3
cp 1
3
[1, 2, 3] 3
cp 2
3
[0, 2, 3] 3
cp 2
3
[0, 2, 3] 3
cp 3
3
[0, 1, 3] 3
(4, 568) 568.0
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [1. 1. 0. ... 1. 1. 1.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 1.]
 [0. 1. 1. ... 1. 0. 0.]]
4 29
568 568
Iteration: 50 - Best solution: 3773.

cp 3
3
[0, 1, 3] 3
cp 0
7
[0, 1, 2, 0, 1, 2, 3] 3
cp 0
7
[0, 1, 2, 0, 1, 2, 3] 

(4, 568) 568.0
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [1. 1. 0. ... 1. 1. 1.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [1. 1. 1. ... 1. 0. 0.]]
4 29
568 568
cp 3
3
[0, 1, 3] 3
cp 2
3
[0, 2, 3] 3
cp 2
3
[0, 2, 3] 3
cp 0
7
[0, 1, 2, 0, 1, 2, 3] 3
cp 1
3
[1, 2, 3] 3
cp 3
3
[0, 1, 3] 3
cp 3
3
[0, 1, 3] 3
cp 3
3
[0, 1, 3] 3
cp 2
3
[0, 2, 3] 3
cp 3
3
[0, 1, 3] 3
(4, 568) 568.0
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [1. 1. 0. ... 1. 1. 1.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [1. 1. 1. ... 1. 0. 0.]]
4 29
568 568
cp 3
3
[0, 1, 3] 3
cp 2
3
[0, 2, 3] 3
cp 0
7
[0, 1, 2, 0, 1, 2, 3] 3
cp 3
3
[0, 1, 3] 3
cp 2
3
[0, 2, 3] 3
cp 3
3
[0, 1, 3] 3
cp 3
3
[0, 1, 3] 3
cp 2
3
[0, 2, 3] 3
cp 2
3
[0, 2, 3] 3
cp 3
3
[0, 1, 3] 3
cp 3
3
[0, 1, 3] 3
(4, 568) 568.0
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [1. 1. 0. ... 1. 1. 1.]
 [0. 0. 0. ... 0. 0. 0.]]
[

cp 2
3
[0, 2, 3] 3
cp 2
3
[0, 2, 3] 3
cp 3
3
[0, 1, 3] 3
cp 3
3
[0, 1, 3] 3
cp 3
3
[0, 1, 3] 3
cp 2
3
[0, 2, 3] 3
cp 3
3
[0, 1, 3] 3
cp 3
3
[0, 1, 3] 3
(4, 568) 568.0
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [1. 1. 0. ... 1. 1. 1.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 1.]
 [1. 1. 1. ... 1. 0. 0.]]
4 29
568 568
cp 3
3
[0, 1, 3] 3
cp 1
3
[1, 2, 3] 3
cp 3
3
[0, 1, 3] 3
cp 3
3
[0, 1, 3] 3
cp 2
3
[0, 2, 3] 3
cp 3
3
[0, 1, 3] 3
cp 0
7
[0, 1, 2, 0, 1, 2, 3] 3
cp 3
3
[0, 1, 3] 3
cp 2
3
[0, 2, 3] 3
(4, 568) 568.0
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [1. 1. 0. ... 1. 1. 1.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 1.]
 [1. 1. 1. ... 1. 0. 0.]]
4 29
568 568
cp 3
3
[0, 1, 3] 3
cp 3
3
[0, 1, 3] 3
cp 3
3
[0, 1, 3] 3
cp 3
3
[0, 1, 3] 3
cp 0
7
[0, 1, 2, 0, 1, 2, 3] 3
cp 3
3
[0, 1, 3] 3
cp 0
7
[0, 1, 2, 0, 1, 2, 3] 3
cp 3
3
[0, 1, 3] 3
cp 2
3
[0, 2, 3] 3
cp 3
3
[0,

...

568 568
cp 3
4
[0, 1, 3, 4] 4
cp 0
9
[0, 1, 2, 3, 0, 1, 2, 3, 4] 4
cp 0
9
[0, 1, 2, 3, 0, 1, 2, 3, 4] 4
cp 0
9
[0, 1, 2, 3, 0, 1, 2, 3, 4] 4
cp 0
9
[0, 1, 2, 3, 0, 1, 2, 3, 4] 4
cp 2
4
[0, 2, 3, 4] 4
cp 4
4
[0, 1, 2, 4] 4
(5, 568) 568.0
[[1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 1. 1.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 1. 1. 1.]]
5 29
568 568
cp 4
4
[0, 1, 2, 4] 4
cp 4
4
[0, 1, 2, 4] 4
cp 0
9
[0, 1, 2, 3, 0, 1, 2, 3, 4] 4
cp 1
4
[1, 2, 3, 4] 4
cp 0
9
[0, 1, 2, 3, 0, 1, 2, 3, 4] 4
cp 0
9
[0, 1, 2, 3, 0, 1, 2, 3, 4] 4
cp 2
4
[0, 2, 3, 4] 4
cp 0
9
[0, 1, 2, 3, 0, 1, 2, 3, 4] 4
(5, 568) 568.0
[[1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 1. 1.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 1. 1. 

(5, 568) 568.0
[[1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 1. 1.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 1. 1. 1.]]
5 29
568 568
cp 4
4
[0, 1, 2, 4] 4
cp 1
4
[1, 2, 3, 4] 4
cp 4
4
[0, 1, 2, 4] 4
cp 0
9
[0, 1, 2, 3, 0, 1, 2, 3, 4] 4
cp 4
4
[0, 1, 2, 4] 4
cp 1
4
[1, 2, 3, 4] 4
(5, 568) 568.0
[[1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 1. 1.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 1. 1. 1.]]
5 29
568 568
cp 3
4
[0, 1, 3, 4] 4
cp 0
9
[0, 1, 2, 3, 0, 1, 2, 3, 4] 4
cp 1
4
[1, 2, 3, 4] 4
cp 0
9
[0, 1, 2, 3, 0, 1, 2, 3, 4] 4
cp 0
9
[0, 1, 2, 3, 0, 1, 2, 3, 4] 4
cp 4
4
[0, 1, 2, 4] 4
cp 4
4
[0, 1, 2, 4] 4
(5, 568) 568.0
[[1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0.

568 568
cp 3
4
[0, 1, 3, 4] 4
cp 4
4
[0, 1, 2, 4] 4
cp 4
4
[0, 1, 2, 4] 4
cp 4
4
[0, 1, 2, 4] 4
(5, 568) 568.0
[[1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 1. 1.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 1. 1. 1.]]
5 29
568 568
cp 0
9
[0, 1, 2, 3, 0, 1, 2, 3, 4] 4
cp 2
4
[0, 2, 3, 4] 4
cp 4
4
[0, 1, 2, 4] 4
cp 1
4
[1, 2, 3, 4] 4
cp 4
4
[0, 1, 2, 4] 4
(5, 568) 568.0
[[1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 1. 1.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 1. 1. 1.]]
5 29
568 568
cp 4
4
[0, 1, 2, 4] 4
cp 4
4
[0, 1, 2, 4] 4
cp 4
4
[0, 1, 2, 4] 4
cp 4
4
[0, 1, 2, 4] 4
cp 4
4
[0, 1, 2, 4] 4
cp 4
4
[0, 1, 2, 4] 4
(5, 568) 568.0
[[1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 1. 0. 0

(5, 568) 568.0
[[1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 1. 1.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 1. 1. ... 1. 1. 1.]]
5 29
568 568
Iteration: 80 - Best solution: 3565.

cp 4
4
[0, 1, 2, 4] 4
cp 1
4
[1, 2, 3, 4] 4
cp 4
4
[0, 1, 2, 4] 4
(5, 568) 568.0
[[1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 1. 1.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 1. 1. ... 1. 1. 1.]]
5 29
568 568
cp 4
4
[0, 1, 2, 4] 4
cp 0
9
[0, 1, 2, 3, 0, 1, 2, 3, 4] 4
cp 4
4
[0, 1, 2, 4] 4
cp 0
9
[0, 1, 2, 3, 0, 1, 2, 3, 4] 4
(5, 568) 568.0
[[1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 1. 1.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [

(5, 568) 568.0
[[1. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 1. 1.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [1. 1. 1. ... 1. 1. 1.]]
5 29
568 568
5 29
568 568
ERROR
1 146
2 3
Summary pile 0 equals pile 0 (appears 1 times), and covers other 78 piles (79 total).
Summary pile 1 equals pile 0 (appears 1 times), and covers other 26 piles (27 total).
Summary pile 2 equals pile 0 (appears 1 times), and covers other 21 piles (22 total).
Summary pile 3 equals pile 0 (appears 1 times), and covers other 43 piles (44 total).
Summary pile 4 equals pile 0 (appears 1 times), and covers other 414 piles (415 total).



In [27]:
a

{'seed': array([[2.],
        [3.]]),
 'minK': 4,
 'maxK': 5,
 'runs': 1,
 'pmj': [[array([0.13458262, 0.04599659, 0.03747871, 0.07495741, 0.70698467])],
  [array([0.13458262, 0.04599659, 0.03747871, 0.07495741, 0.70698467])]],
 'ximli': [[array([0.13458262, 0.04599659, 0.03747871, 0.07495741, 0.70698467])],
  [array([0.13458262, 0.04599659, 0.03747871, 0.07495741, 0.70698467])]],
 'Z': array([[3773.],
        [3565.]]),
 'nIterations': 99,
 'truePos': [[array([0.13458262, 0.04599659, 0.03747871, 0.07495741, 0.70698467])],
  [array([0.13458262, 0.04599659, 0.03747871, 0.07495741, 0.70698467])]],
 'trueNeg': [[array([0.13458262, 0.04599659, 0.03747871, 0.07495741, 0.70698467])],
  [array([0.13458262, 0.04599659, 0.03747871, 0.07495741, 0.70698467])]],
 'covRate': [[array([0.13458262, 0.04599659, 0.03747871, 0.07495741, 0.70698467])],
  [array([0.13458262, 0.04599659, 0.03747871, 0.07495741, 0.70698467])]]}